In [10]:
import sys
import os

# Get the absolute path of the project root
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add 'src/' to Python path
sys.path.append(os.path.join(PROJECT_ROOT, "src"))

# Verify the path
print("Updated sys.path:", sys.path)

Updated sys.path: ['/opt/anaconda3/envs/mlopsproj/lib/python310.zip', '/opt/anaconda3/envs/mlopsproj/lib/python3.10', '/opt/anaconda3/envs/mlopsproj/lib/python3.10/lib-dynload', '', '/Users/pravinut/mlopsproj_py/lib/python3.10/site-packages', '/Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/aiml_orchestrators_assignment/src', '/Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/aiml_orchestrators_assignment/src', '/Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/aiml_orchestrators_assignment/src']


In [11]:
import numpy as np
import idx2numpy
import os

DATA_PATH = "/Users/pravinut/vnit/sem2/mlops/Assignments/Assignment2/data"

def load_mnist_images(filename):
    filepath = os.path.join(DATA_PATH, filename)
    return idx2numpy.convert_from_file(filepath)

def preprocess():
    x_train = load_mnist_images("train-images.idx3-ubyte") / 255.0
    y_train = load_mnist_images("train-labels.idx1-ubyte")
    x_test = load_mnist_images("t10k-images.idx3-ubyte") / 255.0
    y_test = load_mnist_images("t10k-labels.idx1-ubyte")
    return (x_train, y_train), (x_test, y_test)

if __name__ == "__main__":
    preprocess()

In [14]:
import mlflow

# Set MLflow to use local directory
mlflow.set_tracking_uri("file:///Users/pravinut/vnit/sem2/mlops/Assignments/Assignment2/aiml_orchestrators_assignment/mlflow/mlruns")

# Verify tracking URI
print("MLflow Tracking URI:", mlflow.get_tracking_uri())


MLflow Tracking URI: file:///Users/pravinut/vnit/sem2/mlops/Assignments/Assignment2/aiml_orchestrators_assignment/mlflow/mlruns


In [15]:
import tensorflow as tf
import mlflow
import mlflow.tensorflow
import pickle
import joblib
from hwdr_preprocess import preprocess




mlflow.set_experiment("Handwritten Digit Recognition")

(x_train, y_train), (x_test, y_test) = preprocess()

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Example input (for MLflow)
input_example = np.random.rand(1, 28, 28, 1).astype(np.float32)

# Start Experiment Run
with mlflow.start_run():
    model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
    
    test_loss, test_acc = model.evaluate(x_test, y_test)

    # Log Parameters and Metrics
    mlflow.log_param("optimizer", "adam")
    mlflow.log_param("epochs", 5)
    mlflow.log_metric("test_accuracy", test_acc)

    # Log Model with Input Example
    mlflow.tensorflow.log_model(model, "hwdr_model", input_example=input_example)

    # Save using Pickle and Joblib
    pickle.dump(model, open("../models/hwdr_best_model.pkl", "wb"))
    joblib.dump(model, "../models/hwdr_best_model.joblib")

2025/02/17 02:48:54 INFO mlflow.tracking.fluent: Experiment with name 'Handwritten Digit Recognition' does not exist. Creating a new experiment.


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9146 - loss: 0.3002 - val_accuracy: 0.9784 - val_loss: 0.0653
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9833 - loss: 0.0548 - val_accuracy: 0.9846 - val_loss: 0.0483
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9905 - loss: 0.0310 - val_accuracy: 0.9867 - val_loss: 0.0392
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9949 - loss: 0.0179 - val_accuracy: 0.9825 - val_loss: 0.0549
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9956 - loss: 0.0131 - val_accuracy: 0.9861 - val_loss: 0.0434
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9819 - loss: 0.0562 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
